In [1]:
import tensorflow as tf
import pandas as pd
import jieba as jb
import numpy as np

### 1. 数据探索
先用100条的小数据集进行探索：

In [2]:
df = pd.read_csv('./dataset/douban_100.csv', encoding='utf-8')
df.head()

,ID,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,1,Avengers Age of Ultron,复仇者联盟2,2017-01-22,2,更深的白色,2015-04-24,2,非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、...,1231
2,2,Avengers Age of Ultron,复仇者联盟2,2017-01-22,3,有意识的贱民,2015-04-26,2,2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出...,1052
3,3,Avengers Age of Ultron,复仇者联盟2,2017-01-22,4,不老的李大爷耶,2015-04-23,4,《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于...,1045
4,4,Avengers Age of Ultron,复仇者联盟2,2017-01-22,5,ZephyrO,2015-04-22,2,虽然从头打到尾，但是真的很无聊啊。,723


按星级分成正面评论和负面评论两个部分，同时删除各种符号：

In [3]:
def wash(str_original, replace_by_space = True):
    # 要清洗删除的字符
    ls_delete = ['\u3000', '\\n', ',', '。', ':', '：', '，', '“', '”', 
                 '（', '）', '《', '》', '！', '!', '、', '(', ')', '·',
                '.', '？', '…', '>', '「', '」', '|', '\\', '\/', '←',
                '【', '】', '[', ']', '-', '√']
    for k in range(len(ls_delete)):
        # 用空格替代，相当于标点符号处都固定分词，免得后面分词混淆
        if replace_by_space:
            str_original = str(str_original).replace(ls_delete[k], ' ')  
        else:
            str_original = str(str_original).replace(ls_delete[k], '')
    return str_original

def get_stopwords():
    stopwords = [line.strip() for line in open('dataset/stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords

stopwords = get_stopwords()
x = []
y = []

# 遍历所有评论条目，将正面和负面评论存入对应list中
for k in range(len(df['Comment'])): 
    s = wash(df.loc[k, 'Comment'])
    l = [e for e in jb.lcut(s) if (e!=' ' and e not in stopwords)]
    x.append(l)
    if df.loc[k, 'Star'] < 3: 
        y.append(0)
    else:
        y.append(1)
       
x0 = np.array(x)
y0 = np.array(y)
x0[0:3]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.700 seconds.
Prefix dict has been built successfully.


array([list(['奥创', '知道', '整容', '韩国']),
       list(['非常', '失望', '剧本', '完全', '敷衍了事', '主线', '剧情', '没', '突破', '理解', '人物', '缺乏', '动机', '正邪', '之间', '妇联', '内部', '没什么', '火花', '团结', '分裂', '团结', '三段式', '老套', '其实', '利用', '积攒', '下来', '形象', '魅力', '搞', '出', '意思', '剧本', '写得', '非常', '肤浅', '平面', '场面', '调度', '混乱', '呆板', '满屏', '铁甲', '审美疲劳', '笑', '点算', '差强人意']),
       list(['2015', '年度', '失望', '作品', '面面俱到', '实则', '画蛇添足', '主题深刻', '实则', '老调重弹', '推陈出新', '实则', '俗不可耐', '场面', 'high', '实则', 'high', '劲', '不足', '气', '一集', '趣味', '全无', '这集', '笑', '点', '明显', '刻意', '心虚', '全片', '没有', '片段', '紧张', '激动', '太弱', '奥创'])],
      dtype=object)

使用`Keras`的`Tokenizer`类将文本转换成数字。对于中文来说，每个词需要用空格隔开：

In [4]:
def join_text_by_tag(sentences, tag=' '):
    a = []
    for l in sentences:
        a.append(tag.join(str(w) for w in l))
    return a
        
x1 = join_text_by_tag(x0)
x1[0:5]

['奥创 知道 整容 韩国',
 '非常 失望 剧本 完全 敷衍了事 主线 剧情 没 突破 理解 人物 缺乏 动机 正邪 之间 妇联 内部 没什么 火花 团结 分裂 团结 三段式 老套 其实 利用 积攒 下来 形象 魅力 搞 出 意思 剧本 写得 非常 肤浅 平面 场面 调度 混乱 呆板 满屏 铁甲 审美疲劳 笑 点算 差强人意',
 '2015 年度 失望 作品 面面俱到 实则 画蛇添足 主题深刻 实则 老调重弹 推陈出新 实则 俗不可耐 场面 high 实则 high 劲 不足 气 一集 趣味 全无 这集 笑 点 明显 刻意 心虚 全片 没有 片段 紧张 激动 太弱 奥创',
 '铁人 中 勾引 钢铁 侠 妇联 中 勾引 鹰眼 美队 中 勾引 美国 队长 妇联 中 终于 绿巨人 表白 黑寡妇 实际行动 告诉 忠贞不二 治疗 不孕 不育 作战 武器 变成 两支 验孕 棒 坚决 相信 快银 没有 死 后面 回来',
 '从头 打到 尾 真的 无聊']

将文本转换成数字：

In [10]:
tk = tf.keras.preprocessing.text.Tokenizer(char_level=False)
tk.fit_on_texts(x1)
# tk.word_counts
# tk.word_docs
# tk.word_index
x2 = tk.texts_to_sequences(x1)
x2[0:3]

[[1, 98, 369, 178],
 [38,
  21,
  99,
  66,
  370,
  371,
  11,
  12,
  372,
  373,
  6,
  100,
  374,
  375,
  376,
  26,
  377,
  179,
  378,
  180,
  379,
  180,
  380,
  381,
  67,
  382,
  383,
  181,
  384,
  385,
  68,
  69,
  386,
  99,
  387,
  38,
  101,
  388,
  15,
  389,
  182,
  390,
  391,
  392,
  183,
  102,
  393,
  394],
 [395,
  396,
  21,
  397,
  398,
  70,
  399,
  400,
  70,
  401,
  402,
  70,
  403,
  15,
  103,
  70,
  103,
  184,
  404,
  405,
  71,
  185,
  406,
  186,
  102,
  72,
  104,
  105,
  407,
  106,
  2,
  408,
  409,
  187,
  410,
  1]]